In [82]:
import numpy as np
import pandas as pd
from scipy import interpolate as intp
from scipy import optimize
import matplotlib.pyplot as plt
import gsw

In [132]:
def find_sigma0_z(salinity, temperature, pressure, latitude, longitude, sigmas):
    """
    Find the depth of the isopycnal using T/S from a cast
    Inputs:
        salinity    (array) : Salinity in PSU
        temperature (array) : In-situ temperature (C)
        latitude    (array) : Latitude of the sample
        longitude   (array) : Longitude of the sample
        sigmas      (array) : Isopycnals for which to find the depth
    Outputs:
        sigma_z     (array) : Depth of the isopycnals
    Algorithm:
        1. Convert in-situ salinity and temperature to Absolute Salinity and Conservative Temperature
        2. Calculate sigma0
        3. Check to ensure the isopycnal surface spans the density range of the water column
        4. Build interpolating functions for both T and S
    """    
    
    def dens_diff(pressure, SA_f, CT_f, target_sigma0):
        return np.square(gsw.sigma0(SA_f(pressure),CT_f(pressure)) - target_sigma0)
    
    # Promote to single element array if only one level requested
    
    is_scalar = type(sigmas) == type(0.)
    if is_scalar:
        sigmas = np.array([sigmas])
    
    pressure = np.array(pressure)
    SA = gsw.SA_from_SP(salinity, pressure, longitude, latitude)
    CT = gsw.CT_from_t(salinity, temperature, pressure)
    
    P_sort = np.unique(pressure)
    SA_sort = np.zeros(P_sort.shape)
    CT_sort = np.zeros(P_sort.shape)
    # Loop through all pressures that overlap, average the temperatures and salinity accordingly
    for idx, P in enumerate(P_sort):
        presidx = (pressure == P)        
        SA_sort[idx] = SA[presidx].mean()
        CT_sort[idx] = CT[presidx].mean()        
    
    SA_intp = intp.interp1d(P_sort, SA_sort, kind='quadratic')
    CT_intp = intp.interp1d(P_sort, CT_sort, kind='quadratic')
        
    sigma0 = gsw.sigma0(SA_sort,CT_sort)
    sigma0_max = sigma0.max()
    sigma0_min = sigma0.min()
    
    sigma0_z = np.zeros(len(sigmas))
    
    for sigidx, siglev in enumerate(sigmas):
        if (siglev < sigma0_min) or (siglev > sigma0_max):
            sigma0_z[sigidx] = np.nan
        else:
            for botidx in range(len(sigma0)-1):
                if (siglev >= sigma0[botidx]) & (siglev <= sigma0[botidx+1]):
                    start_idx = botidx
                    break
                                              
            out = optimize.minimize_scalar(dens_diff,
                                            bounds=[P_sort[botidx],P_sort[botidx+1]],
                                            method='Bounded',
                                            args=(SA_intp, CT_intp, siglev))
            
            sigma0_z[sigidx] = out.x

    if is_scalar:
        sigma0_z = np.squeeze(sigma0_z)
    
    return sigma0_z
    
    
    

In [126]:
path = '/home/ashao/data/glodap/'
glodap = pd.read_csv(path+'GLODAPv2.2019_Merged_Master_File.csv')
cruise_ids = glodap.cruise.unique()
glodap_by_cruise = [ glodap[(glodap.cruise == cruise_id) & (glodap.salinityqc == 1)] for cruise_id in cruise_ids ]

In [127]:
cruise_df = glodap_by_cruise[0]

In [139]:
def process_cruise(cruise_df,sigma0_vals):
    nsigma0 = len(sigma0_vals)
    stations = cruise_df.station.unique()
    cols = ['latitude','longitude'] + [ f'{sig0}_z' for sig0 in sigma0_vals ]   
    df_out = pd.DataFrame(columns = cols)
    for station in stations:
        df = cruise_df[ cruise_df.station == station ]
        sigma0_z = find_sigma0_z(df.salinity,df.temperature,df.pressure,df.latitude,df.longitude,sigma0_vals)
        out = np.nan*np.ones(nsigma0+2)
        out[0] = df.latitude.mean()
        out[1] = df.longitude.mean()
        out[2:] = sigma0_z        
        df_out = df_out.append( pd.DataFrame([out], columns = cols) )
    
    return df_out
    

In [150]:
out = process_cruise(cruise_df,np.arange(26,27.1,0.1))



In [151]:
out

,latitude,longitude,26.0_z,26.1_z,26.200000000000003_z,26.300000000000004_z,26.400000000000006_z,26.500000000000007_z,26.60000000000001_z,26.70000000000001_z,26.80000000000001_z,26.900000000000013_z,27.000000000000014_z,27.100000000000016_z
0,80.567,7.2267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.694537,13.583451,16.606930,19.783198
0,80.633,9.4600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.971914,8.439523,11.051453,13.836842,16.836197
0,80.733,12.8530,NaN,NaN,5.121509,7.155966,9.244169,11.388475,13.591835,15.858033,18.192005,20.600247,23.091437,25.677345
0,80.905,18.5880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,81.052,17.6680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.996887,9.413883,12.044532
0,81.197,16.7930,4.031728,5.014592,6.027652,7.073494,8.155112,9.275983,10.440211,11.652656,12.919195,14.247009,15.645048,17.124710
0,81.308,15.3850,NaN,NaN,NaN,NaN,NaN,NaN,6.034357,8.407920,10.961304,13.726002,16.745955,20.087136
0,81.767,-10.6430,24.634554,38.990713,43.595117,47.012148,49.857537,52.517108,55.253891,58.075845,60.991581,64.011242,67.146956,70.413408
0,81.837,-10.4900,21.129828,43.001721,52.542118,59.320435,64.237235,68.217316,71.650801,74.714758,77.507397,80.089904,82.503467,84.779980
0,81.907,-10.2470,25.427137,52.958892,64.428778,71.038468,76.219193,80.626871,84.530944,88.073452,91.340163,94.461630,97.736062,101.220612


In [128]:
cruises = glodap.cruise.unique()
# This will be looped over cruises
cruise_df  = glodap[glodap.cruise == cruises[0]]
# This will be a loop over stations
stations = cruise_df.station.unique()
station_df = cruise_df[ cruise_df.station == stations[0] ]

In [121]:
test = pd.DataFrame(columns=['lat','lon','27.1'])

In [11]:
SA = gsw.SA_from_SP(station_df.salinity, station_df.pressure, station_df.longitude, station_df.latitude)
CT = gsw.CT_from_t(station_df.salinity,station_df.temperature,station_df.pressure)
sigma0 = gsw.sigma0(SA,CT)

In [111]:
test = pd.DataFrame(data = [[1,1]],columns=['Lat','Lon'])
test

,Lat,Lon
0,1,1


In [131]:
find_sigma0_z(station_df.salinity,station_df.temperature,station_df.pressure,station_df.latitude,station_df.longitude,27.)

23.993230253379235
33.50676974662076
18.113539493241525
14.841146828463938
16.647081356363486
16.62810924676955
16.60617636963789
16.60692328955262
16.606929812877517
16.606933392531428


array(16.60692981)

In [115]:
station_df.append?

Signature:
station_df.append(
    other,
    ignore_index=False,
    verify_integrity=False,
    sort=None,
)
Docstring:
Append rows of `other` to the end of caller, returning a new object.

Columns in `other` that are not in the caller are added as new columns.

Parameters
----------
other : DataFrame or Series/dict-like object, or list of these
    The data to append.
ignore_index : boolean, default False
    If True, do not use the index labels.
verify_integrity : boolean, default False
    If True, raise ValueError on creating index with duplicates.
sort : boolean, default None
    Sort columns if the columns of `self` and `other` are not aligned.
    The default sorting is deprecated and will change to not-sorting
    in a future version of pandas. Explicitly pass ``sort=True`` to
    silence the warning and sort. Explicitly pass ``sort=False`` to
    silence the warning and not sort.

    .. versionadded:: 0.23.0

Returns
-------
DataFrame

See Also
--------
concat : General func